# Welcome to **metal_complex**

## What can achieve this python package?

### Initialisation

First of all, the functions must be import.

In [ ]:
import sys
import os
notebook_path = os.getcwd()  # Get the current working directory
src_path = os.path.abspath(os.path.join(notebook_path, "../src"))
sys.path.insert(0, src_path)
from main_functions import *

### Modelise 3D metal complex
This package has few useful functions as a chemist. First of all, let's talk about the 3D vizualisation of a metal complex as said in the package name. The next code show a six coordination number complex: $[Fe(en)_{3}]^{+3}$

In [10]:
list_of_ligand_SMILES = ['N([H])([H])C([H])([H])C([H])([H])N([H])[H]','N([H])([H])C([H])([H])C([H])([H])N([H])[H]','N([H])([H])C([H])([H])C([H])([H])N([H])[H]']
metal_SMILES = '[Fe]'
metal_complex(list_of_ligand_SMILES, metal_SMILES)

#### Errors
As you can see, the molecule is interactive but some atoms are not recognize by rdkit (`error UFFTYPER`) but it does not impact the viewer. However, this function does not work with all ligands: firstly, it can return a non possible molecule with bad geometry and secondly, the results may changed if the code is use two times:

In [13]:
list_of_ligand_SMILES = ['[Cl]','[Cl]','[Cl]','[Cl]','[Cl]','[Cl]']
metal_SMILES = '[Ni]'
metal_complex(list_of_ligand_SMILES, metal_SMILES)

In this example, the molecule is not octahedric and the bonds between Ni and the Cls are of different length.  
Nevertheless it is not the only problem: in the used data, <u>ligands_misc_info.csv</u>, some ligands does not have filled data. For example, the methane does not appear in the data so it returns a wrong 3D molecule.

In [20]:
list_of_ligand_SMILES = ['C','C','C']
metal_SMILES = '[Fe]'
metal_complex(list_of_ligand_SMILES, metal_SMILES)

### Give the oxydation state

### Give the moleculare mass

A simple function add to the package determine the mass of the 3D complex by giving the SMILES of all the ligands and the one of the metal.  
The functions is round at 3 decimals and show the molecular mass even if the molecule does not appear correctly with the <u>metal_complex</u> function.

In [24]:
list_of_ligand_SMILES = ['[Cl]','[Cl]','[Cl]','[Cl]','[Cl]','[Cl]']
metal_SMILES = '[Ni]'
print(calculate_MO(list_of_ligand_SMILES, metal_SMILES))

list_of_ligand_SMILES = ['c1ccccc1','c1ccccc1','c1ccccc1','c1ccccc1']
metal_SMILES = '[Pt]'
print(calculate_MO(list_of_ligand_SMILES, metal_SMILES))

## Tkinter interface